In [1]:
from elasticsearch import Elasticsearch

In [6]:
client = Elasticsearch(
    basic_auth=("user", "password"),
    cloud_id="cloud_id"
)
health = client.cluster.health()

print(health)
assert health['status'] == "green"

{'cluster_name': '45311c8ec9484c108f38d56fa8feb5a7', 'status': 'green', 'timed_out': False, 'number_of_nodes': 3, 'number_of_data_nodes': 2, 'active_primary_shards': 77, 'active_shards': 154, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}
